https://data.mendeley.com/datasets/3t9vbwxgr5/2/files/d2c58546-d8be-4d57-af14-b61337c927b2

In [3]:
from IPython import display

from utils import Logger

import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("tcc_ceds_music.csv", index_col=0)
df["genre"].value_counts()

pop        7042
country    5445
blues      4604
rock       4034
jazz       3845
reggae     2498
hip hop     904
Name: genre, dtype: int64

In [5]:
pop_df = df[df["genre"] == "pop"]["lyrics"].reset_index(drop=True)


In [18]:
from functools import reduce
t = list(map(set, reduce(lambda x, y: x + y, pop_df.str.split(" "))))

In [17]:
t

NameError: name 't' is not defined